---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [181]:

def graph_identification():
    """
    # Visualization
    import matplotlib.pyplot as plt

    fig, axs = plt.subplots(len(P1_Graphs), figsize=(5, 10))
    for index, graph in enumerate(P1_Graphs):
        frq, edges = np.histogram(list(graph.degree().values()))
        axs[index].bar(edges[:-1], frq, width=np.diff(edges), label=str(index))
        #print("{}: {} {} {}".format(index, nx.average_clustering(graph), nx.average_shortest_path_length(graph), degree_dist))
        axs[index].legend()
    """
    return ["PA", "SW_L", "SW_L", "PA", "SW_H"]

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [58]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))



Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [103]:
def salary_predictions():
    # PREPROCESSING
    df = pd.DataFrame(index=G.nodes())

    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))

    train_df = df[~df["ManagementSalary"].isnull()]
    test_df = df[df["ManagementSalary"].isnull()]

    X = train_df.drop("ManagementSalary", axis=1)
    y = train_df["ManagementSalary"].astype(bool)
    
    # CLASSIFICATION
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score, confusion_matrix

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

    gbc = GradientBoostingClassifier().fit(X_train, y_train)
    
    return pd.Series(index=test_df.index, data=gbc.predict_proba(test_df.drop("ManagementSalary", axis=1))[:,1])

salary_predictions()

1       0.012632
2       0.980451
5       0.992252
8       0.078216
14      0.067011
18      0.032886
27      0.024365
30      0.887191
31      0.260695
34      0.013392
37      0.032469
40      0.026618
45      0.007195
54      0.561450
55      0.872918
60      0.075126
62      0.991779
65      0.988443
77      0.009851
79      0.008567
97      0.004502
101     0.001508
103     0.499353
108     0.015692
113     0.030593
122     0.001526
141     0.113013
142     0.991779
144     0.014920
145     0.801820
          ...   
913     0.005956
914     0.010749
915     0.001336
918     0.006826
923     0.019003
926     0.033541
931     0.004230
934     0.000770
939     0.000974
944     0.000931
945     0.019003
947     0.021859
950     0.014408
951     0.003064
953     0.002853
959     0.000974
962     0.000974
963     0.022253
968     0.011855
969     0.011855
974     0.019190
984     0.000770
987     0.019455
989     0.021859
991     0.020457
992     0.000974
994     0.000671
996     0.0010

In [97]:
"""# PREPROCESSING
df = pd.DataFrame(index=G.nodes())

df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df['clustering'] = pd.Series(nx.clustering(G))
df['degree'] = pd.Series(G.degree())
df['betweenness'] = pd.Series(nx.betweenness_centrality(G))
df['closeness'] = pd.Series(nx.closeness_centrality(G))

train_df = df[~df["ManagementSalary"].isnull()]
test_df = df[df["ManagementSalary"].isnull()]

X = train_df.drop("ManagementSalary", axis=1)
y = train_df["ManagementSalary"].astype(bool)"""

In [85]:
# X

In [56]:
"""# VISUALIZATION
import matplotlib.pyplot as plt
from matplotlib import cm

cmap = cm.get_cmap("gnuplot")
scatter = pd.scatter_matrix(X, c=y, marker="o", s=40, hist_kwds={"bins":15}, figsize=(12,12), cmap=cmap)"""

'# VISUALIZATION\nimport matplotlib.pyplot as plt\nfrom matplotlib import cm\n\ncmap = cm.get_cmap("gnuplot")\nscatter = pd.scatter_matrix(X, c=y, marker="o", s=40, hist_kwds={"bins":15}, figsize=(12,12), cmap=cmap)'

In [98]:
"""# CLASSIFICATION
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

gbc = GradientBoostingClassifier().fit(X_train, y_train)"""

In [99]:
"""from sklearn.model_selection import cross_val_score

test_gbc = GradientBoostingClassifier()
cv_scores = cross_val_score(test_gbc, X, y, scoring="roc_auc")
cv_scores"""

array([ 0.96709906,  0.95      ,  0.93875319])

In [100]:
"""# EVALUATION
y_scores = gbc.decision_function(X_test)
roc_auc_score(y_test, y_scores)"""

0.919601677148847

In [101]:
"""# RESULT
pd.Series(index=test_df.index, data=gbc.predict_proba(test_df.drop("ManagementSalary", axis=1))[:,1])"""

1      -4.358773
2       3.915099
5       4.852567
8      -2.466843
14     -2.633529
18     -3.381259
27     -3.689959
30      2.062364
31     -1.042357
34     -4.299608
37     -3.394464
40     -3.599188
45     -4.927174
54     -0.005356
55      1.927007
60     -2.510489
62      4.792787
65      4.448880
77     -4.610236
79     -4.751277
97     -5.398640
101    -6.495759
103    -0.002589
108    -4.138771
113    -3.455912
122    -6.483415
141    -2.060332
142     4.792787
144    -4.189987
145     1.397709
          ...   
913    -5.117318
914    -4.522144
915    -6.616714
918    -4.980203
923    -3.943977
926    -3.360880
931    -5.461266
934    -7.167804
939    -6.933063
944    -6.978218
945    -3.943977
947    -3.801019
950    -4.225460
951    -5.784825
953    -5.856434
959    -6.933063
962    -6.933063
963    -3.778583
968    -4.423072
969    -4.423072
974    -3.933991
984    -7.167804
987    -3.920028
989    -3.801019
991    -3.868739
992    -6.933063
994    -7.305963
996    -6.8809

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [104]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [149]:
def new_connections_predictions():
    # PREPROCESSING
    copy_df = future_connections.copy()
    copy_df["common_neighbors"] = [len(list(nx.common_neighbors(G, connection[0], connection[1]))) for connection in copy_df.index]
    copy_df["jaccard_coefficient"] = np.array(list(nx.jaccard_coefficient(G, copy_df.index)))[:,2]
    copy_df["resource_allocation_index"] = np.array(list(nx.resource_allocation_index(G, copy_df.index)))[:,2]
    copy_df["preferential_attachment"] = np.array(list(nx.preferential_attachment(G, copy_df.index)))[:,2]
    copy_df["cn_soundarajan_hopcroft"] = np.array(list(nx.cn_soundarajan_hopcroft(G, copy_df.index, community="Department")))[:,2]
    copy_df["ra_index_soundarajan_hopcroft"] = np.array(list(nx.ra_index_soundarajan_hopcroft(G, copy_df.index, community="Department")))[:,2]

    train_df = copy_df[~copy_df["Future Connection"].isnull()]
    test_df = copy_df[copy_df["Future Connection"].isnull()]

    X = train_df.drop("Future Connection", axis=1)
    y = train_df["Future Connection"].astype(bool)
    
    # CLASSIFICATION
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score, confusion_matrix

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

    gbc = GradientBoostingClassifier().fit(X_train, y_train)
    
    return pd.Series(index=test_df.index, data=gbc.predict_proba(test_df.drop("Future Connection", axis=1))[:,1])

new_connections_predictions()

(107, 348)    0.024775
(542, 751)    0.012984
(20, 426)     0.592178
(50, 989)     0.012984
(942, 986)    0.012984
(324, 857)    0.012984
(13, 710)     0.139933
(19, 271)     0.152634
(319, 878)    0.012984
(659, 707)    0.012984
(49, 843)     0.012984
(208, 893)    0.012984
(377, 469)    0.015796
(405, 999)    0.014596
(129, 740)    0.013850
(292, 618)    0.014596
(239, 689)    0.012984
(359, 373)    0.015620
(53, 523)     0.379821
(276, 984)    0.012984
(202, 997)    0.012984
(604, 619)    0.042029
(270, 911)    0.012984
(261, 481)    0.060756
(200, 450)    0.997039
(213, 634)    0.012984
(644, 735)    0.033279
(346, 553)    0.012984
(521, 738)    0.012984
(422, 953)    0.016052
                ...   
(672, 848)    0.012984
(28, 127)     0.974579
(202, 661)    0.012984
(54, 195)     0.997024
(295, 864)    0.012984
(814, 936)    0.012984
(839, 874)    0.012984
(139, 843)    0.012984
(461, 544)    0.014282
(68, 487)     0.014282
(622, 932)    0.012984
(504, 936)    0.016800
(479, 528) 

In [140]:
"""# PREPROCESSING
copy_df = future_connections.copy()
copy_df["common_neighbors"] = [len(list(nx.common_neighbors(G, connection[0], connection[1]))) for connection in copy_df.index]
copy_df["jaccard_coefficient"] = np.array(list(nx.jaccard_coefficient(G, copy_df.index)))[:,2]
copy_df["resource_allocation_index"] = np.array(list(nx.resource_allocation_index(G, copy_df.index)))[:,2]
copy_df["preferential_attachment"] = np.array(list(nx.preferential_attachment(G, copy_df.index)))[:,2]
copy_df["cn_soundarajan_hopcroft"] = np.array(list(nx.cn_soundarajan_hopcroft(G, copy_df.index, community="Department")))[:,2]
copy_df["ra_index_soundarajan_hopcroft"] = np.array(list(nx.ra_index_soundarajan_hopcroft(G, copy_df.index, community="Department")))[:,2]

train_df = copy_df[~copy_df["Future Connection"].isnull()]
test_df = copy_df[copy_df["Future Connection"].isnull()]

X = train_df.drop("Future Connection", axis=1)
y = train_df["Future Connection"].astype(bool)"""

In [145]:
"""# CLASSIFICATION
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

gbc = GradientBoostingClassifier().fit(X_train, y_train)"""

In [146]:
"""# EVALUATION
y_scores = gbc.decision_function(X_test)
roc_auc_score(y_test, y_scores)"""

0.91415062033464589

In [148]:
"""# RESULT
pd.Series(index=test_df.index, data=gbc.predict_proba(test_df.drop("Future Connection", axis=1))[:,1])"""

(107, 348)    0.024775
(542, 751)    0.012984
(20, 426)     0.592178
(50, 989)     0.012984
(942, 986)    0.012984
(324, 857)    0.012984
(13, 710)     0.139933
(19, 271)     0.152634
(319, 878)    0.012984
(659, 707)    0.012984
(49, 843)     0.012984
(208, 893)    0.012984
(377, 469)    0.015796
(405, 999)    0.014596
(129, 740)    0.013850
(292, 618)    0.014596
(239, 689)    0.012984
(359, 373)    0.015620
(53, 523)     0.379821
(276, 984)    0.012984
(202, 997)    0.012984
(604, 619)    0.042029
(270, 911)    0.012984
(261, 481)    0.060756
(200, 450)    0.997039
(213, 634)    0.012984
(644, 735)    0.033279
(346, 553)    0.012984
(521, 738)    0.012984
(422, 953)    0.016052
                ...   
(672, 848)    0.012984
(28, 127)     0.974579
(202, 661)    0.012984
(54, 195)     0.997024
(295, 864)    0.012984
(814, 936)    0.012984
(839, 874)    0.012984
(139, 843)    0.012984
(461, 544)    0.014282
(68, 487)     0.014282
(622, 932)    0.012984
(504, 936)    0.016800
(479, 528) 